In [1]:
import pyodbc
import tensorflow as tf
from joblib import load
import pandas as pd
import numpy as np
import urllib
import sqlalchemy

In [2]:

# Connect to MSSQL database

driver= '{ODBC Driver 17 for SQL Server}'

connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=localhost;DATABASE=AnimeInsight;UID=sa;PWD=12345678;Trusted_Connection=yes')

cursor = connection.cursor()


In [3]:

df = pd.read_sql('SELECT * FROM anime_metadata', connection)


C:\Users\aunno\AppData\Local\Temp\ipykernel_14344\4234207955.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT * FROM anime_metadata', connection)


In [4]:
connection.close()

In [5]:
df

,anime_id,Name,English_name,Score,Genres,Synopsis,Type,Episodes,Aired,Premiered,Status,Producers,Licensors,Studios,Source,Duration,Age_Rating,Rank,Scored_By,Image_URL
0,1,Cowboy Bebop,Cowboy Bebop,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",spring 1998,Finished Airing,Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),41.0,914193.0,https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",None,Finished Airing,"Sunrise, Bandai Visual",Sony Pictures Entertainment,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),189.0,206248.0,https://cdn.myanimelist.net/images/anime/1439/...
2,6,Trigun,Trigun,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",spring 1998,Finished Airing,Victor Entertainment,"Funimation, Geneon Entertainment USA",Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,328.0,356739.0,https://cdn.myanimelist.net/images/anime/7/203...
3,7,Witch Hunter Robin,Witch Hunter Robin,7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,26.0,"Jul 3, 2002 to Dec 25, 2002",summer 2002,Finished Airing,"Bandai Visual, Dentsu, Victor Entertainment, T...","Funimation, Bandai Entertainment",Sunrise,Original,25 min per ep,PG-13 - Teens 13 or older,2764.0,42829.0,https://cdn.myanimelist.net/images/anime/10/19...
4,8,Bouken Ou Beet,Beet the Vandel Buster,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,52.0,"Sep 30, 2004 to Sep 29, 2005",fall 2004,Finished Airing,"TV Tokyo, Dentsu",Illumitoon Entertainment,Toei Animation,Manga,23 min per ep,PG - Children,4240.0,6413.0,https://cdn.myanimelist.net/images/anime/7/215...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24900,55731,Wu Nao Monu,None,NaN,"Comedy, Fantasy, Slice of Life",No description available for this anime.,ONA,15.0,"Jul 4, 2023 to ?",None,Not yet aired,None,None,None,Web manga,None,PG-13 - Teens 13 or older,NaN,NaN,https://cdn.myanimelist.net/images/anime/1386/...
24901,55732,Bu Xing Si: Yuan Qi,Blader Soul,NaN,"Action, Adventure, Fantasy",No description available for this anime.,ONA,18.0,"Jul 27, 2023 to ?",None,Not yet aired,None,None,None,Web novel,None,PG-13 - Teens 13 or older,0.0,NaN,https://cdn.myanimelist.net/images/anime/1383/...
24902,55733,Di Yi Xulie,The First Order,NaN,"Action, Adventure, Fantasy, Sci-Fi",No description available for this anime.,ONA,16.0,"Jul 19, 2023 to ?",None,Finished Airing,None,None,None,Web novel,None,PG-13 - Teens 13 or older,0.0,NaN,https://cdn.myanimelist.net/images/anime/1130/...
24903,55734,Bokura no Saishuu Sensou,None,NaN,None,A music video for the song Bokura no Saishuu S...,Music,1.0,"Apr 23, 2022",None,Finished Airing,None,None,None,Original,3 min,PG-13 - Teens 13 or older,0.0,NaN,https://cdn.myanimelist.net/images/anime/1931/...


In [6]:

# Load your TensorFlow model
model = tf.keras.models.load_model('models/anime_model_2.keras')


In [7]:
model.get_weights()

[array([[ 5.62825380e-03,  9.91345104e-03, -1.10392766e-02,
          9.54613090e-03,  4.28834930e-02, -5.84538188e-03,
          5.27394190e-03,  2.14367732e-02, -1.09464033e-02,
          1.99142732e-02, -1.10869920e-02,  9.95585322e-03,
          3.48102488e-02, -1.63817797e-02,  3.46202813e-02,
          5.19492722e-04],
        [-3.36411744e-02, -4.52087522e-02,  1.22640021e-02,
          1.69651620e-02,  1.74281485e-02, -1.01978891e-02,
         -2.52470374e-02,  4.39139642e-02,  2.92385109e-02,
         -1.03427880e-02,  3.15418877e-02,  1.45793445e-02,
         -3.49535830e-02,  2.66151503e-03, -3.76952291e-02,
          1.00169070e-02],
        [-1.13230109e-01, -1.83368567e-02, -1.95923582e-01,
          1.08080685e-01,  7.57813826e-02, -5.30283935e-02,
          1.81446657e-01,  6.98020458e-02,  1.48174046e-02,
         -1.41629711e-01, -5.26574105e-02,  4.28185165e-02,
          1.16613664e-01,  7.89154544e-02, -1.12268880e-01,
          3.13983522e-02],
        [ 1.0995087

In [8]:
mean_score_scaler=load('preprocessing/mean_score_scaler.pkl')

rating_scaler=load('preprocessing/rating_scaler.pkl')

score_scaler=load('preprocessing/score_scaler.pkl')

type_encoder=load('preprocessing/Type_encoder.pkl')


gender_encoder=load('preprocessing/gender_encoder.pkl')

user_input_shape=6

anime_input_shape=8

studio_max_length = 10

genre_max_length=9

source_max_length=1

In [9]:
@tf.keras.utils.register_keras_serializable()
def split_func(input_str):
  return tf.strings.split(input_str, sep=", ")

In [10]:
studio_vectorize_layer_model=tf.keras.models.load_model('preprocessing/studio_vectorize_layer_model')
studio_vectorize_layer = studio_vectorize_layer_model.layers[0]

num_studios=len(studio_vectorize_layer.get_vocabulary())
num_studios

914

In [11]:
genre_vectorize_layer_model=tf.keras.models.load_model('preprocessing/genre_vectorize_layer_model')
genre_vectorize_layer = genre_vectorize_layer_model.layers[0]

num_genres=len(genre_vectorize_layer.get_vocabulary())
num_genres

23

In [12]:
source_vectorize_layer_model=tf.keras.models.load_model('preprocessing/source_vectorize_layer_model')
source_vectorize_layer = source_vectorize_layer_model.layers[0]


num_sources=len(source_vectorize_layer.get_vocabulary())
num_sources

19

In [13]:
df.drop(['English_name',"Name",'Synopsis','Age_Rating','Rank','Scored_By','Status','Premiered','Episodes','Duration','Licensors','Producers' , 'Aired','Image_URL'],axis=1,inplace=True)

In [14]:
df.fillna({'Score':df['Score'].mean()},inplace=True)

In [15]:
df.loc[:,'Score']=score_scaler.transform(df[['Score']]).astype(np.float32)

c:\Users\aunno\.conda\envs\db-project\lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but PowerTransformer was fitted without feature names
  warnings.warn(


In [16]:
encoded_Types = type_encoder.transform(df[['Type']])  # Fit and transform the column
encoded_df = pd.DataFrame(encoded_Types.toarray(), columns=type_encoder.categories_[0],index=df.index)  # Create a DataFrame from the encoded columns

df=pd.concat([df, encoded_df], axis=1)  # Concatenate the original DataFrame with the encoded DataFrame
df.drop(['Type'],axis=1,inplace=True)

In [17]:
df.fillna({'Studios':'','Genres':'','Source':''},inplace=True)

In [18]:
df

,anime_id,Score,Genres,Studios,Source,Movie,Music,ONA,OVA,Special,TV,UNKNOWN
0,1,2.657709,"Action, Award Winning, Sci-Fi",Sunrise,Original,0,0,0,0,0,1,0
1,5,2.225084,"Action, Sci-Fi",Bones,Original,1,0,0,0,0,0,0
2,6,2.039589,"Action, Adventure, Sci-Fi",Madhouse,Manga,0,0,0,0,0,1,0
3,7,0.936342,"Action, Drama, Mystery, Supernatural",Sunrise,Original,0,0,0,0,0,1,0
4,8,0.591776,"Adventure, Fantasy, Supernatural",Toei Animation,Manga,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
24900,55731,-0.019334,"Comedy, Fantasy, Slice of Life",,Web manga,0,0,1,0,0,0,0
24901,55732,-0.019334,"Action, Adventure, Fantasy",,Web novel,0,0,1,0,0,0,0
24902,55733,-0.019334,"Action, Adventure, Fantasy, Sci-Fi",,Web novel,0,0,1,0,0,0,0
24903,55734,-0.019334,,,Original,0,1,0,0,0,0,0


In [19]:
sequences=studio_vectorize_layer(df.loc[:,'Studios'])
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=studio_max_length, padding='post')

# Create separate columns for each index in the sequence
for i in range(studio_max_length):
    df[f'Studio_Index_{i+1}'] = padded_sequences[:, i]

df.drop('Studios', axis=1, inplace=True)

In [20]:
sequences=genre_vectorize_layer(df.loc[:,'Genres'])

padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=genre_max_length, padding='post')

# Create separate columns for each index in the sequence
for i in range(genre_max_length):
    df[f'Genre_Index_{i+1}'] = padded_sequences[:, i]

df.drop('Genres', axis=1, inplace=True)

In [21]:
sequences=source_vectorize_layer(df.loc[:,'Source'])

padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=source_max_length, padding='post')

# Create separate columns for each index in the sequence
for i in range(source_max_length):
    df[f'Source_Index_{i+1}'] = padded_sequences[:, i]

df.drop('Source', axis=1, inplace=True)

In [22]:
anime_columns=df.iloc[:,1:1+anime_input_shape].to_numpy(dtype='float32')
anime_studio_embedding_columns = df.iloc[:,1+anime_input_shape:1+anime_input_shape+studio_max_length].to_numpy()
anime_genre_embedding_columns= df.iloc[:,1+anime_input_shape+studio_max_length:1+anime_input_shape+studio_max_length+genre_max_length].to_numpy()
anime_source_embedding_columns = df.iloc[:,-1:].to_numpy()

In [23]:
input={'anime_inputs':anime_columns,'anime_genre_embeddings_inputs':anime_genre_embedding_columns,'anime_studio_embeddings_inputs':anime_studio_embedding_columns,'anime_source_embeddings_inputs':anime_source_embedding_columns}

In [24]:
output=model(input)

In [25]:
output_df = pd.DataFrame(output,columns=['tensor_1', 'tensor_2', 'tensor_3', 'tensor_4', 'tensor_5', 'tensor_6', 'tensor_7', 'tensor_8', 'tensor_9', 'tensor_10', 'tensor_11', 'tensor_12', 'tensor_13', 'tensor_14', 'tensor_15', 'tensor_16'])

In [26]:
final_df =pd.concat([df['anime_id'], output_df], axis=1)

In [27]:
final_df.rename(columns={'anime_id':'AnimeID'},inplace=True)

In [28]:
final_df

,AnimeID,tensor_1,tensor_2,tensor_3,tensor_4,tensor_5,tensor_6,tensor_7,tensor_8,tensor_9,tensor_10,tensor_11,tensor_12,tensor_13,tensor_14,tensor_15,tensor_16
0,1,0.0,0.110924,0.000000,0.0,0.0,0.0,0.0,0.0,0.840022,0.0,0.0,0.544059,0.0,0.000000,0.0,0.0
1,5,0.0,0.575299,0.000000,0.0,0.0,0.0,0.0,0.0,1.210965,0.0,0.0,0.587626,0.0,0.000000,0.0,0.0
2,6,0.0,0.668368,0.000000,0.0,0.0,0.0,0.0,0.0,1.285309,0.0,0.0,0.596358,0.0,0.000000,0.0,0.0
3,7,0.0,1.762228,0.150932,0.0,0.0,0.0,0.0,0.0,2.132592,0.0,0.0,0.695576,0.0,0.000000,0.0,0.0
4,8,0.0,3.118280,0.783984,0.0,0.0,0.0,0.0,0.0,2.798486,0.0,0.0,0.270716,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24900,55731,0.0,3.620287,1.217431,0.0,0.0,0.0,0.0,0.0,3.008914,0.0,0.0,0.126145,0.0,0.000000,0.0,0.0
24901,55732,0.0,4.094989,1.537540,0.0,0.0,0.0,0.0,0.0,3.119853,0.0,0.0,0.000000,0.0,0.536808,0.0,0.0
24902,55733,0.0,4.079476,1.585809,0.0,0.0,0.0,0.0,0.0,3.114585,0.0,0.0,0.000000,0.0,0.548875,0.0,0.0
24903,55734,0.0,2.479611,0.353573,0.0,0.0,0.0,0.0,0.0,2.623366,0.0,0.0,0.659284,0.0,0.000000,0.0,0.0


In [29]:

connect_string = urllib.parse.quote_plus(f'DRIVER={{ODBC Driver 17 for SQL Server}};Server=localhost;Database=AnimeInsight;UID=sa;PWD=12345678;Trusted_Connection=yes')
engine = sqlalchemy.create_engine(f'mssql+pyodbc:///?odbc_connect={connect_string}', fast_executemany=True)

with engine.connect() as connection:
  final_df.to_sql('anime_tensors', connection, index=False, if_exists='append')